In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [31]:
from lightfm import LightFM
from rectools.metrics import MAP
from rectools.models import LightFMWrapperModel
from rectools import Columns
from rectools.dataset import Dataset
from rectools.tools import UserToItemAnnRecommender
from pathlib import Path
import pandas as pd
import nmslib

## Loading data

In [4]:
DATA_PATH = Path("../../data/03_primary")

In [5]:
users_features_full = pd.read_csv(DATA_PATH / 'au_featured_users_full.csv')
items_features_full = pd.read_csv(DATA_PATH / 'prepared_featured_items_full.csv')
interactions_full = pd.read_csv(DATA_PATH / 'au_interactions_full.csv')
items_descriptions = pd.read_csv(DATA_PATH / 'items_descriptions_cleared.csv')

In [6]:
full_dataset = Dataset.construct(
    interactions_df=interactions_full,
)

full_featured_dataset = Dataset.construct(
    interactions_df=interactions_full,
    user_features_df=users_features_full,
    cat_user_features=["sex", "age", "income"],
    item_features_df=items_features_full,
    cat_item_features=["genre", "content_type","age_rating","country","release_year_bin"],
)

In [7]:
# IDs from data preparation notebook
artificial_users_ids = [1097559, 1097561, 1097563]

## Models training

All hyperparameters was taken from LightFM notebook

In [8]:
K_RECOS = 10
RANDOM_STATE = 42
NUM_THREADS = 4

In [9]:
lightfm_without_features = LightFMWrapperModel(
    model = LightFM(
        no_components = 14,
        learning_schedule = 'adadelta',
        loss = 'warp',
        rho = 0.9293803074906476,
        epsilon = 1.019065155861173e-06,
        learning_rate = 0.0012055115615607931,
        random_state = RANDOM_STATE
    )
)

lightfm_without_features.fit(full_dataset)

In [10]:
lightfm_with_features = LightFMWrapperModel(
    model = LightFM(
        no_components = 5,
        learning_schedule = 'adadelta',
        loss = 'warp',
        rho = 0.9479133091288102,
        epsilon = 3.434929518317852e-06,
        learning_rate = 0.0007143586518906775,
        random_state = RANDOM_STATE
    )
)

lightfm_with_features.fit(full_featured_dataset)

In [11]:
user_embeddings, item_embeddings = lightfm_with_features.get_vectors(full_featured_dataset)

In [34]:
ann = UserToItemAnnRecommender(
    user_vectors=user_embeddings,
    item_vectors=item_embeddings,
    user_id_map=full_featured_dataset.user_id_map,
    item_id_map=full_featured_dataset.item_id_map,
    index_init_params={
        'method' : 'hnsw',
        'space' : 'negdotprod',
        'data_type' : nmslib.DataType.DENSE_VECTOR
    }
    
)

In [35]:
ann.fit()

## Checking recommendations

1097559 - Mother with kid

1097561 - Superheroics fan

1097563 - Born in USSR

### LightFM without features

In [14]:
lightfm_wo_features_reco = lightfm_without_features.recommend(
    users=artificial_users_ids,
    dataset=full_dataset,
    k=K_RECOS,
    filter_viewed=True,
)

In [15]:
lightfm_wo_features_reco = lightfm_wo_features_reco.merge(items_descriptions, on='item_id')\
    .sort_values(by=['user_id','rank'])

#### Recos for mother with kid

In [16]:
lightfm_wo_features_reco[:10]

,user_id,item_id,score,rank,content_type,title,release_year,genres,countries,age_rating
0,1097559,15297,5.036095,1,series,Клиника счастья,2021.0,"драмы, мелодрамы",Россия,18.0
3,1097559,13865,4.983968,2,film,Девятаев,2021.0,"драмы, военные, приключения",Россия,12.0
6,1097559,10440,4.978795,3,series,Хрустальный,2021.0,"триллеры, детективы",Россия,18.0
9,1097559,9728,4.839619,4,film,Гнев человеческий,2021.0,"боевики, триллеры","Великобритания, США",18.0
12,1097559,4151,4.774676,5,series,Секреты семейной жизни,2021.0,комедии,Россия,18.0
15,1097559,2657,4.591525,6,series,Подслушано,2021.0,"драмы, триллеры",Россия,16.0
18,1097559,3734,4.567730,7,film,Прабабушка легкого поведения,2021.0,комедии,Россия,16.0
21,1097559,4880,4.350189,8,series,Афера,2021.0,комедии,Россия,18.0
24,1097559,8636,4.302714,9,film,Белый снег,2021.0,"драмы, спорт",Россия,6.0
26,1097559,9996,4.288709,10,series,Немцы,2021.0,драмы,Россия,16.0


#### Recos for superheroics fan

In [17]:
lightfm_wo_features_reco[10:20]

,user_id,item_id,score,rank,content_type,title,release_year,genres,countries,age_rating
1,1097561,15297,5.024084,1,series,Клиника счастья,2021.0,"драмы, мелодрамы",Россия,18.0
4,1097561,13865,4.992881,2,film,Девятаев,2021.0,"драмы, военные, приключения",Россия,12.0
7,1097561,10440,4.971634,3,series,Хрустальный,2021.0,"триллеры, детективы",Россия,18.0
10,1097561,9728,4.835921,4,film,Гнев человеческий,2021.0,"боевики, триллеры","Великобритания, США",18.0
13,1097561,4151,4.744411,5,series,Секреты семейной жизни,2021.0,комедии,Россия,18.0
19,1097561,3734,4.552613,6,film,Прабабушка легкого поведения,2021.0,комедии,Россия,16.0
16,1097561,2657,4.546454,7,series,Подслушано,2021.0,"драмы, триллеры",Россия,16.0
22,1097561,4880,4.316914,8,series,Афера,2021.0,комедии,Россия,18.0
28,1097561,142,4.271284,9,film,Маша,2020.0,"драмы, триллеры",Россия,16.0
25,1097561,8636,4.268815,10,film,Белый снег,2021.0,"драмы, спорт",Россия,6.0


#### Recos for old man

In [18]:
lightfm_wo_features_reco[20:]

,user_id,item_id,score,rank,content_type,title,release_year,genres,countries,age_rating
2,1097563,15297,5.076678,1,series,Клиника счастья,2021.0,"драмы, мелодрамы",Россия,18.0
5,1097563,13865,5.003299,2,film,Девятаев,2021.0,"драмы, военные, приключения",Россия,12.0
8,1097563,10440,4.991167,3,series,Хрустальный,2021.0,"триллеры, детективы",Россия,18.0
11,1097563,9728,4.877642,4,film,Гнев человеческий,2021.0,"боевики, триллеры","Великобритания, США",18.0
14,1097563,4151,4.774202,5,series,Секреты семейной жизни,2021.0,комедии,Россия,18.0
17,1097563,2657,4.616301,6,series,Подслушано,2021.0,"драмы, триллеры",Россия,16.0
20,1097563,3734,4.578351,7,film,Прабабушка легкого поведения,2021.0,комедии,Россия,16.0
23,1097563,4880,4.358158,8,series,Афера,2021.0,комедии,Россия,18.0
29,1097563,142,4.345122,9,film,Маша,2020.0,"драмы, триллеры",Россия,16.0
27,1097563,9996,4.303714,10,series,Немцы,2021.0,драмы,Россия,16.0


### LightFM with features

In [19]:
lightfm_with_features_reco = lightfm_with_features.recommend(
    users=artificial_users_ids,
    dataset=full_featured_dataset,
    k=K_RECOS,
    filter_viewed=True,
)

In [20]:
lightfm_with_features_reco = lightfm_with_features_reco.merge(items_descriptions, on='item_id')\
    .sort_values(by=['user_id','rank'])

#### Recos for mother with kid

In [21]:
lightfm_with_features_reco[:10]

,user_id,item_id,score,rank,content_type,title,release_year,genres,countries,age_rating
0,1097559,15297,-75639.990310,1,series,Клиника счастья,2021.0,"драмы, мелодрамы",Россия,18.0
3,1097559,10440,-75640.255096,2,series,Хрустальный,2021.0,"триллеры, детективы",Россия,18.0
6,1097559,2657,-75640.452532,3,series,Подслушано,2021.0,"драмы, триллеры",Россия,16.0
9,1097559,9728,-75640.595357,4,film,Гнев человеческий,2021.0,"боевики, триллеры","Великобритания, США",18.0
12,1097559,4151,-75640.752990,5,series,Секреты семейной жизни,2021.0,комедии,Россия,18.0
15,1097559,3734,-75640.803709,6,film,Прабабушка легкого поведения,2021.0,комедии,Россия,16.0
18,1097559,142,-75641.031472,7,film,Маша,2020.0,"драмы, триллеры",Россия,16.0
21,1097559,14431,-75641.097455,8,film,Приворот. Чёрное венчание,2021.0,"ужасы, триллеры, мелодрамы",Россия,16.0
22,1097559,13865,-75641.166346,9,film,Девятаев,2021.0,"драмы, военные, приключения",Россия,12.0
25,1097559,16228,-75641.239085,10,series,Содержанки,2021.0,триллеры,Россия,18.0


#### Recos for superheroins fan

In [22]:
lightfm_with_features_reco[10:20]

,user_id,item_id,score,rank,content_type,title,release_year,genres,countries,age_rating
10,1097561,9728,-100659.113713,1,film,Гнев человеческий,2021.0,"боевики, триллеры","Великобритания, США",18.0
4,1097561,10440,-100659.269912,2,series,Хрустальный,2021.0,"триллеры, детективы",Россия,18.0
1,1097561,15297,-100659.322444,3,series,Клиника счастья,2021.0,"драмы, мелодрамы",Россия,18.0
7,1097561,2657,-100659.387607,4,series,Подслушано,2021.0,"драмы, триллеры",Россия,16.0
13,1097561,4151,-100659.456974,5,series,Секреты семейной жизни,2021.0,комедии,Россия,18.0
16,1097561,3734,-100659.586491,6,film,Прабабушка легкого поведения,2021.0,комедии,Россия,16.0
23,1097561,13865,-100659.594226,7,film,Девятаев,2021.0,"драмы, военные, приключения",Россия,12.0
26,1097561,4880,-100659.608468,8,series,Афера,2021.0,комедии,Россия,18.0
28,1097561,9996,-100659.921411,9,series,Немцы,2021.0,драмы,Россия,16.0
19,1097561,142,-100659.968842,10,film,Маша,2020.0,"драмы, триллеры",Россия,16.0


#### Recos for old man

In [23]:
lightfm_with_features_reco[20:]

,user_id,item_id,score,rank,content_type,title,release_year,genres,countries,age_rating
24,1097563,13865,-93634.704321,1,film,Девятаев,2021.0,"драмы, военные, приключения",Россия,12.0
5,1097563,10440,-93634.750726,2,series,Хрустальный,2021.0,"триллеры, детективы",Россия,18.0
2,1097563,15297,-93634.750928,3,series,Клиника счастья,2021.0,"драмы, мелодрамы",Россия,18.0
11,1097563,9728,-93634.765182,4,film,Гнев человеческий,2021.0,"боевики, триллеры","Великобритания, США",18.0
8,1097563,2657,-93635.150461,5,series,Подслушано,2021.0,"драмы, триллеры",Россия,16.0
14,1097563,4151,-93635.182490,6,series,Секреты семейной жизни,2021.0,комедии,Россия,18.0
29,1097563,9996,-93635.193251,7,series,Немцы,2021.0,драмы,Россия,16.0
27,1097563,4880,-93635.209838,8,series,Афера,2021.0,комедии,Россия,18.0
17,1097563,3734,-93635.263568,9,film,Прабабушка легкого поведения,2021.0,комедии,Россия,16.0
20,1097563,142,-93635.403161,10,film,Маша,2020.0,"драмы, триллеры",Россия,16.0


### ANN

In [36]:
ann_reco = ann.get_item_list_for_user_batch(
    user_ids=artificial_users_ids,
    top_n=K_RECOS
)

In [37]:
ann_recos_df = pd.DataFrame({
    Columns.User: artificial_users_ids,
    'reco': ann_reco
})

In [38]:
ann_recos_df = ann_recos_df\
    .set_index('user_id')\
    .explode('reco')\
    .rename(columns={
        'reco': 'item_id'
    }).reset_index()\
    .merge(items_descriptions, on='item_id')\
    .sort_values(by=['user_id'])

#### Recos for mother with kid

In [39]:
ann_recos_df[:10]

,user_id,item_id,content_type,title,release_year,genres,countries,age_rating
0,1097559,14163,film,Королева клуба,2020.0,"драмы, комедии",Канада,18.0
18,1097559,6143,film,Версальский роман,2014.0,мелодрамы,Великобритания,16.0
17,1097559,3991,film,Король сафари,2013.0,"мультфильм, приключения, комедии",ЮАР,0.0
15,1097559,7033,series,Обними меня,2014.0,"мелодрамы, комедии",Россия,16.0
11,1097559,13202,film,Наруто: Последний фильм,2014.0,"боевики, аниме, приключения, комедии",Япония,12.0
10,1097559,602,film,Чем заняться дома?,2020.0,"мультфильм, комедии",Кипр,0.0
9,1097559,13653,film,102 далматинца,2000.0,"семейное, комедии","США, Великобритания",16.0
14,1097559,10469,film,Три богатыря и Морской царь,2016.0,"мультфильм, приключения, комедии",Россия,6.0
6,1097559,5635,film,Другая женщина,2014.0,"мелодрамы, комедии",США,16.0
3,1097559,5831,film,Поезд в Пусан,2016.0,"боевики, ужасы, триллеры",Республика Корея,18.0


#### Recos for superheroins fan

In [40]:
ann_recos_df[10:20]

,user_id,item_id,content_type,title,release_year,genres,countries,age_rating
1,1097561,14163,film,Королева клуба,2020.0,"драмы, комедии",Канада,18.0
27,1097561,7439,film,Эскобар,2018.0,"драмы, биография, криминал","Испания, Болгария, США",18.0
4,1097561,5831,film,Поезд в Пусан,2016.0,"боевики, ужасы, триллеры",Республика Корея,18.0
12,1097561,13202,film,Наруто: Последний фильм,2014.0,"боевики, аниме, приключения, комедии",Япония,12.0
25,1097561,11710,film,Игры разумов,2018.0,"драмы, биография","Ирландия, Франция, Исландия, США, Мексика, Бел...",16.0
16,1097561,7033,series,Обними меня,2014.0,"мелодрамы, комедии",Россия,16.0
24,1097561,15894,film,Синяя бездна 2,2019.0,"драмы, ужасы, триллеры, приключения","Великобритания, США",16.0
7,1097561,5635,film,Другая женщина,2014.0,"мелодрамы, комедии",США,16.0
20,1097561,1962,film,Время первых,2017.0,"биография, триллеры, приключения, историческое",Россия,6.0
22,1097561,2558,film,Добро пожаловать в капкан,2013.0,"боевики, триллеры, криминал","Великобритания, США",16.0


#### Recos for old man

In [41]:
ann_recos_df[20:]

,user_id,item_id,content_type,title,release_year,genres,countries,age_rating
26,1097563,11710,film,Игры разумов,2018.0,"драмы, биография","Ирландия, Франция, Исландия, США, Мексика, Бел...",16.0
23,1097563,2558,film,Добро пожаловать в капкан,2013.0,"боевики, триллеры, криминал","Великобритания, США",16.0
8,1097563,5635,film,Другая женщина,2014.0,"мелодрамы, комедии",США,16.0
19,1097563,6143,film,Версальский роман,2014.0,мелодрамы,Великобритания,16.0
2,1097563,14163,film,Королева клуба,2020.0,"драмы, комедии",Канада,18.0
28,1097563,15421,film,Мужчина и женщина: Лучшие годы,2019.0,драмы,Франция,16.0
13,1097563,13202,film,Наруто: Последний фильм,2014.0,"боевики, аниме, приключения, комедии",Япония,12.0
5,1097563,5831,film,Поезд в Пусан,2016.0,"боевики, ужасы, триллеры",Республика Корея,18.0
21,1097563,1962,film,Время первых,2017.0,"биография, триллеры, приключения, историческое",Россия,6.0
29,1097563,7599,film,Звёздные войны: Возвращение Джедая,1983.0,"боевики, фантастика, приключения",США,0.0


## Conclusion

LightFM keeps trying and trying to push popular items into the recommendations. ANN gave us some kind of trash